In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qiskit.aqua import QuantumInstance
from qiskit import BasicAer
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE, IQPE
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import ExcitationPreserving

from qiskit.circuit.library import ExcitationPreserving

from qiskit.chemistry.core import ChemistryOperator

from qiskit import BasicAer
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.aqua.algorithms import VQAlgorithm, VQE, MinimumEigensolver
from qiskit.chemistry.qubit_transformations import FermionicTransformation
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.ground_state_calculation import MinimumEigensolverGroundStateCalculation
from qiskit.chemistry.extrapolator import *

In [2]:
# import BOPES
from qiskit.chemistry.bopes_sampler import BOPESSampler

In [3]:
ft = FermionicTransformation()
driver = PySCFDriver()
solver = VQE(quantum_instance=
             QuantumInstance(backend=BasicAer.get_backend('statevector_simulator')))
#solver = NumPyMinimumEigensolver()
me_gsc = MinimumEigensolverGroundStateCalculation(ft, solver)

In [4]:
# import molecule 
from qiskit.chemistry.drivers.molecule import Molecule
from functools import partial

In [5]:
stretch1 = partial(Molecule.absolute_stretching, atom_pair=(1, 0))
mol = Molecule(geometry=[('H', [0., 0., 0.]),
                        ('H', [0., 0., 0.3])],
                       degrees_of_freedom=[stretch1],
                       )

# pass molecule to PSYCF driver
driver = PySCFDriver(molecule=mol)

In [6]:
# BOPES sampler testing
bs = BOPESSampler(gsc=me_gsc, driver=driver, 
                  bootstrap=True, num_bootstrap=3)
points = np.linspace(0.1,3,8)
res = bs.compute_surface(points)

In [9]:
print(res)

In [15]:
# Extrapolate testing

extrap_poly = Extrapolator.factory("poly", kwargs={'degree': 2})
extrapolator = Extrapolator.factory("window", kwargs={'extrapolator': extrap_poly})
window_extrapolator = Extrapolator.factory("window",
                                           extrapolator=extrapolator)
bs_extr = BOPESSampler(gsc=me_gsc, driver=driver, 
                  bootstrap=True, num_bootstrap=3
                 , extrapolator = window_extrapolator
                 )
res = bs_extr.compute_surface(points)

TypeError: __init__() got an unexpected keyword argument 'kwargs'

In [ ]:
energies = []
energies_extr = []
for point in points:
    energy = bs_res_full[point]['computed_electronic_energy'] + bs_res_full[point]['nuclear_repulsion_energy']
    energy_extr = bs_res_full_extr[point]['computed_electronic_energy'] + bs_res_full_extr[point]['nuclear_repulsion_energy']
    energies.append(energy)
    energies_extr.append(energy_extr)

In [ ]:
fig = plt.figure()
plt.plot(points,energies,label = 'bootstrapping')
plt.plot(points,energies_extr,label = 'bootstrapping with extrap')
plt.legend()
plt.title('Dissociation profile')
plt.xlabel('Interatomic distance')
plt.ylabel('Energy')

In [ ]:
bs = bs_res_full_extr
for key in bs:
    print(bs[key]['raw_result']['cost_function_evals'])

In [ ]:
from qiskit.chemistry.energy_surface_spline import EnergySurface1DSpline
energy_surface = EnergySurface1DSpline()

# fit data
xdata = bs_res['point']
ydata = bs_res['energy']
energy_surface.fit_to_data(xdata=xdata, ydata=ydata)

# newx = np.unique(xdata)
# newy = [np.average(ydata[np.where(xdata== val)[0][0]])
#          for val in np.unique(xdata)]
# print(newx)
# print(newy)
# print(xdata)
# print(ydata)
# # for val in np.unique(xdata):
# #     #print(val)
# #     print(np.where(xdata == val)[0])

In [ ]:
plt.plot(xdata, ydata, 'kx')
x = np.arange(min(xdata) - 0.25, max(xdata) + 0.25, 0.05)
plt.plot(x, energy_surface.eval(x), 'r-')
plt.xlabel(r'distance, $\AA$')
plt.ylabel('energy, Hartree')
dist = max(ydata) - min(ydata)
plt.ylim(min(ydata) - 0.1 * dist, max(ydata) + 0.1 * dist)